<h1>Land Feature CNN</h1>

Code and network framework gathered from:
https://www.tensorflow.org/tutorials/images/cnn

Authors:
- Thao Pham
- Lawrence Hoerst


In [ ]:
import converter
# This will load the colored images into np arrays and pickle them
converter.main()

In [4]:
# Import tensorflow for the model
import tensorflow as tf
# Import matplotlib to show retults
import matplotlib.pyplot as plt
# Import pickle to read in training data
import pickle

2.10.0


In [ ]:
# Class names to plot the images:
class_names = ['Water', 'Buildings', 'Roads', 'Foliage', 'Mineral deposits', 'Mountainous terrain', 'Rocky terrain', 'Sandy terrain', 'Plains', 'Snow', 'Grass']
class_colors = ['#0f5e9c', ('#f2f2f2', '#606060'), '#c4c4c4', '#3a5f0b', '#490e0e', '#5a7a4c', '#698287', '#f7ae64', '#c89e23', '#fffafa', '#7cfc00']
class_codes = {class_names[i]: i for i in range(len(class_names))}

# hex_to_rgb
cvt = lambda hex: ImageColor.getcolor(hex, "RGB")
colors_to_code = {cvt('#0f5e9c'): 0,
                  cvt('#f2f2f2'): 1, cvt('#606060'): 1,
                  cvt('#c4c4c4'): 2,
                  cvt('#3a5f0b'): 3,
                  cvt('#490e0e'): 4,
                  cvt('#5a7a4c'): 5,
                  cvt('#698287'): 6,
                  cvt('#f7ae64'): 7,
                  cvt('#c89e23'): 8,
                  cvt('#fffafa'): 9,
                  cvt('#7cfc00'): 10,
                  'default':      11}

In [2]:
# Instantiate the model
model = models.Sequential()
# Add layers
# We need to decide on how many layers we want
# These parameters are not yet setup for our network
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(200*200, activation='relu'))
model.add(layers.Dense(200*200))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

NameError: name 'models' is not defined

In [1]:
print(model.summary())

NameError: name 'model' is not defined

In [3]:
# unpickle our test data here
with open('training_inputs.pickle', 'rb') as inputsFile:
    training_inputs = pickle.load(inputsFile)
with open('training_outputs.pickle', 'rb') as outputsFile:
    training_outputs = pickle.load(outputsFile)
    
# TODO: unpickle the validation / testing outputs


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (20, 200, 200) + inhomogeneous part.

In [ ]:
# Train the model
# TODO: We need to add our validation inputs/outputs here
history = model.fit(training_inputs, training_outputs, epochs=10)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)